In [13]:
# meanshift
import numpy as np 
import cv2 

# 读入视频
videoname = "assets/slow_traffic_small.mp4"
cap = cv2.VideoCapture(videoname)
if cap.isOpened() == False:
    print("Error opening video stream or file")
    exit(-1)

ret, frame = cap.read()

# 初始化窗口
x, y, w, h = 300, 200, 100, 50 
track_window = (x, y, w, h)

# 计算直方图
roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# 设置终止条件，10次，精度为1个像素
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while(True):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # 执行均值漂移
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # 画结果
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        cv2.imshow('meanshift', img2)

        
        k = cv2.waitKey(50) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

In [14]:
# CAMshift
import numpy as np
import cv2

# 读入视频
videoname = "assets/slow_traffic_small.mp4"
cap = cv2.VideoCapture(videoname)
if cap.isOpened() == False:
    print("Error opening video stream or file")
    exit(-1)

ret, frame = cap.read()

# 初始化窗口
x, y, w, h = 300, 200, 100, 50
track_window = (x, y, w, h)

# 计算直方图
roi = frame[y:y+h, x:x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60., 32.)),
                   np.array((180., 255., 255.)))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# 设置终止条件，10次，精度为1个像素
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while (True):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # 执行均值漂移
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # 画结果
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, 255, 2)
        cv2.imshow('camshift', img2)

        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()
